# Simulateur basique de l'impôt sur les revenus 2020

In [ ]:
def calcul_impots(revenus, parts_fiscales, frais_reels):
    _BAREME_2020 = [
        {'max': 10064, 'taux': 0/100},
        {'max': 25659, 'taux': 11/100},
        {'max': 73369, 'taux': 30/100},
        {'max': 157806, 'taux': 41/100},
        {'max': float('Inf'), 'taux': 45/100}
    ]
    _SEUIL_RECOUVREMENT = 61  # En-dessous de ce montant, pas d'impôt
    _DEDUCTION_FORFAITAIRE = 10/100  # Décote de 10% forfaitaire
    _SEUIL_DECOTE_CELIBATAIRE = 1711
    _SEUIL_DECOTE_COUPLE = 2842


    if frais_reels <= (revenus * _DEDUCTION_FORFAITAIRE):
        revenus_normalises = revenus * (1 - _DEDUCTION_FORFAITAIRE) / parts_fiscales
    else:
        revenus_normalises = (revenus - frais_reels) / parts_fiscales


    montant_impots = 0
    revenus_restants = revenus_normalises
    max_tranche_precedente = 0

    for tranche in _BAREME_2020:
        
        if revenus_restants >= tranche.get('max') - max_tranche_precedente:
            revenus_tranche = tranche.get('max') - max_tranche_precedente
        else:
            revenus_tranche = revenus_restants
        
        max_tranche_precedente = tranche.get('max')

        revenus_restants -= revenus_tranche
        taux = tranche['taux']

        impots_dans_tranche = revenus_tranche * taux * parts_fiscales
        montant_impots += impots_dans_tranche

        #if tranche.get('taux') != 0:
        #    print(f'Tranche {taux:.0%}: {impots_dans_tranche:.2f}€')
        
        if revenus_restants == 0:
            break

    # print('Impots avant decote: {:.2f} €'.format(montant_impots))

    if parts_fiscales == 1 and montant_impots < _SEUIL_DECOTE_CELIBATAIRE:
        decote = 777 - (45.25/100 * montant_impots)
    elif parts_fiscales >= 2 and montant_impots < _SEUIL_DECOTE_COUPLE:
        decote = 1286 - (45.25/100 * montant_impots)
    else:
        decote = 0

    # print(f'Decote: {decote:.2f} €')

    montant_impots -= decote

    montant_impots = int(montant_impots)

    if montant_impots <= _SEUIL_RECOUVREMENT:
        montant_impots = 0

    return montant_impots

In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider

def print_impots(*args, **kwargs):
    montant_impots = calcul_impots(*args, **kwargs)
    print(f'Montant impots: {montant_impots} €')

interact(
    print_impots,
    revenus=IntSlider(min=0, max=200000, step=100, value=33600),
    parts_fiscales=FloatSlider(min=1, max=5, step=0.5, value=1),
    frais_reels=IntSlider(min=0, max=40000, step=10, value=0),
)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

fig = go.Figure()
fig.update_layout(
    title='Impôts sur les revenus 2020',
    xaxis_title='Revenus en euros',
    yaxis_title='Montant des impôts en euros',
    autosize=False,
    width=1000,
    height=800,
)
fig.update_yaxes(dtick=1000)

fig2 = go.Figure()
fig2.update_layout(
    title='Impôts sur les revenus 2020',
    xaxis_title='Revenus en euros',
    yaxis_title='Taux en %',
    autosize=False,
    width=1000,
    height=800,
)
fig2.update_yaxes(dtick=1)

for parts_fiscales in (1, 1.5, 2, 2.5, 3, 4):
    df = pd.DataFrame()
    df['revenus'] = range(0, 100500, 500)
    df['montant_impots'] = df['revenus'].apply(calcul_impots, parts_fiscales=parts_fiscales, frais_reels=0)
    df['taux_imposition'] = df['montant_impots']/df['revenus'] * 100

    fig.add_trace(
    go.Scatter(
        x=df['revenus'],
        y=df['montant_impots'],
        name=f'{parts_fiscales} parts'
    ))
    fig2.add_trace(
    go.Scatter(
        x=df['revenus'],
        y=df['taux_imposition'],
        name=f'{parts_fiscales} parts'
    ))

fig.show()
fig2.show()